In [1]:
import gradio as gr

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [2]:
def predict_type_NN(flav_text, type_1, type_2, type_3):
    df = pd.read_csv("pokemon_text_cleaned.csv")
    subset = pd.DataFrame({'DexNumber':df['DexNumber'], 'Name':df['Name'], 'FlavorText':df['FlavorText']})

    RAND_STATE = 13
    MAX_ITERATIONS = 700

    if (type_1 == type_2) | (type_1 == type_3) | (type_2 == type_3):
        return {predicted_type: "Error: Input types must be different!", predicted_confidence: str(-1)}

    classes = [type_1, type_2, type_3]

    for i in classes:
        subset[i] = df[i]

    test_array = np.zeros_like(np.asarray(subset[classes[0]]))
    for i,name in enumerate(classes):
        test_array = test_array + np.asarray(subset[name])
    subset = subset[pd.Series(test_array) == 1]

    target_array = np.zeros_like(np.asarray(subset[classes[0]]))
    for i,name in enumerate(classes):
        target_array = target_array + (i)*np.asarray(subset[name])
    
    features = np.asarray(subset['FlavorText'])
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = model.encode(features)
    
    classifier = MLPClassifier(random_state=RAND_STATE, hidden_layer_sizes=[600], max_iter=MAX_ITERATIONS).fit(embeddings, target_array)

    input_sample = model.encode(flav_text).reshape(1,-1)

    type_prediction = classifier.predict(input_sample)[0]

    probs = classifier.predict_proba(input_sample)[0]
    #print(classes[type_prediction])
    #print(probs[type_prediction])
    
    return {predicted_type: classes[type_prediction], predicted_confidence: str(probs[type_prediction])}

In [3]:
def predict_type_NB(flav_text, type_1, type_2, type_3):
    df = pd.read_csv("pokemon_text_cleaned.csv")
    subset = pd.DataFrame({'DexNumber':df['DexNumber'], 'Name':df['Name'], 'FlavorText':df['FlavorText']})

    if (type_1 == type_2) | (type_1 == type_3) | (type_2 == type_3):
        return {predicted_type: "Error: Input types must be different!", predicted_confidence: str(-1)}

    classes = [type_1, type_2, type_3]

    for i in classes:
        subset[i] = df[i]

    test_array = np.zeros_like(np.asarray(subset[classes[0]]))
    for i,name in enumerate(classes):
        test_array = test_array + np.asarray(subset[name])
    subset = subset[pd.Series(test_array) == 1]

    target_array = np.zeros_like(np.asarray(subset[classes[0]]))
    for i,name in enumerate(classes):
        target_array = target_array + (i)*np.asarray(subset[name])
    
    features = np.asarray(subset['FlavorText'])
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = model.encode(features)
    
    classifier = GaussianNB().fit(embeddings, target_array)

    input_sample = model.encode(flav_text).reshape(1,-1)

    type_prediction = classifier.predict(input_sample)[0]

    probs = classifier.predict_proba(input_sample)[0]
    #print(classes[type_prediction])
    #print(probs[type_prediction])
    
    return {predicted_type: classes[type_prediction], predicted_confidence: str(probs[type_prediction])}

In [4]:
def predict_type_LogReg(flav_text, type_1, type_2, type_3):
    df = pd.read_csv("pokemon_text_cleaned.csv")
    subset = pd.DataFrame({'DexNumber':df['DexNumber'], 'Name':df['Name'], 'FlavorText':df['FlavorText']})

    RAND_STATE = 13
    MAX_ITERATIONS = 700
    C_VAL = 8

    if (type_1 == type_2) | (type_1 == type_3) | (type_2 == type_3):
        return {predicted_type: "Error: Input types must be different!", predicted_confidence: str(-1)}

    classes = [type_1, type_2, type_3]

    for i in classes:
        subset[i] = df[i]

    test_array = np.zeros_like(np.asarray(subset[classes[0]]))
    for i,name in enumerate(classes):
        test_array = test_array + np.asarray(subset[name])
    subset = subset[pd.Series(test_array) == 1]

    target_array = np.zeros_like(np.asarray(subset[classes[0]]))
    for i,name in enumerate(classes):
        target_array = target_array + (i)*np.asarray(subset[name])
    
    features = np.asarray(subset['FlavorText'])
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = model.encode(features)
    
    classifier = LogisticRegression(max_iter=MAX_ITERATIONS, random_state=RAND_STATE, solver='sag', C=C_VAL, class_weight='balanced').fit(embeddings, target_array)

    input_sample = model.encode(flav_text).reshape(1,-1)

    type_prediction = classifier.predict(input_sample)[0]

    probs = classifier.predict_proba(input_sample)[0]
    #print(classes[type_prediction])
    #print(probs[type_prediction])
    
    return {predicted_type: classes[type_prediction], predicted_confidence: str(probs[type_prediction])}

In [5]:
with gr.Blocks() as demo:
    with gr.Column() as predictor:
        flav_text = gr.Text(label="Description")
        type_1 = gr.Dropdown(['Normal', 'Fire', 'Water', 'Grass', 'Electric', 'Psychic', 'Ice', 'Dragon', 'Dark', 'Fairy', 'Fighting', 'Flying', 'Poison', 'Ground', 'Rock', 'Bug', 'Ghost', 'Steel'], label="Type 1")
        type_2 = gr.Dropdown(['Normal', 'Fire', 'Water', 'Grass', 'Electric', 'Psychic', 'Ice', 'Dragon', 'Dark', 'Fairy', 'Fighting', 'Flying', 'Poison', 'Ground', 'Rock', 'Bug', 'Ghost', 'Steel'], label="Type 2")
        type_3 = gr.Dropdown(['Normal', 'Fire', 'Water', 'Grass', 'Electric', 'Psychic', 'Ice', 'Dragon', 'Dark', 'Fairy', 'Fighting', 'Flying', 'Poison', 'Ground', 'Rock', 'Bug', 'Ghost', 'Steel'], label="Type 3")
        with gr.Row() as predictors:
            predict_button_NN = gr.Button("Predict With Neural Network")
            predict_button_NB = gr.Button("Predict With Naive Bayes")
            predict_button_LogReg = gr.Button("Predict With Logistic Regression")
            
        predicted_type = gr.Text(label="Predicted Type")
        predicted_confidence = gr.Text(label="Prediction Confidence")

        predict_button_NN.click(fn=predict_type_NN, inputs=[flav_text,type_1,type_2,type_3], outputs=[predicted_type,predicted_confidence])
        predict_button_NB.click(fn=predict_type_NB, inputs=[flav_text,type_1,type_2,type_3], outputs=[predicted_type,predicted_confidence])
        predict_button_LogReg.click(fn=predict_type_LogReg, inputs=[flav_text,type_1,type_2,type_3], outputs=[predicted_type,predicted_confidence])

demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
